In [31]:
# 载入需要用的库
import cifar10, cifar10_input
import tensorflow as tf
import numpy as np
import math
import time

In [34]:
batch_size = 128
data = './cifar10_data/cifar-10-batches-bin'
images_train, labels_train = cifar10_input.inputs(eval_data = False, batch_size = batch_size)
images_test, labels_test = cifar10_input.inputs(eval_data = True, batch_size = batch_size)

In [35]:
def weight_variable(shape, stddev):
    initial = tf.truncated_normal(shape, stddev=stddev)  # stddev=stddev！！！
    return tf.Variable(initial)


def bias_variable(cons, shape):
    initial = tf.constant(cons, shape=shape)  # 必须是 shape=shape
    return tf.Variable(initial)


def conv(x, W):
    return tf.nn.conv2d(x, W, [1, 1, 1, 1], padding='SAME')


def max_pool_3x3(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')


In [36]:
image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])
label_holder = tf.placeholder(tf.int32, [batch_size])

In [37]:
# 第一层
weight1 = weight_variable([5, 5, 3, 64], 5e-2)
bias1 = bias_variable(0.0, [64])

conv1 = tf.nn.relu(conv(image_holder, weight1) + bias1)
pool1 = max_pool_3x3(conv1)

# 第二层
weight2 = weight_variable([5, 5, 64, 64], 5e-2)
bias2 = bias_variable(0.1, [64])

conv2 = tf.nn.relu(conv(pool1, weight2) + bias2)
pool2 = max_pool_3x3(conv2)

reshape = tf.reshape(pool2, [batch_size, -1])
dim = reshape.get_shape()[1].value

# 全连接层
weight3 = weight_variable([dim, 384], 0.04)
bias3 = bias_variable(0.1, [384])

local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)

# 全连接层
weight4 = weight_variable([384, 192], 0.04)
bias4 = bias_variable(0.1, [192])

local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

# 输出
weight5 = weight_variable([192, 10], 1 / 192.0)
bias5 = bias_variable(0.0, [10])
logits = tf.matmul(local4, weight5) + bias5

In [38]:
# 损失函数
def loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels,
                                                                   name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'), name='total_loss')


loss = loss(logits, label_holder)
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.


W0611 21:03:46.343130  6876 deprecation.py:323] From <ipython-input-38-cfd0881e242f>:17: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


W0611 21:03:46.349115  6876 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.


[]

In [40]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
tf.train.start_queue_runners()

max_steps = 3000
for step in range(max_steps):
    start_time = time.time()
    image_batch, label_batch = sess.run([images_train, labels_train])
    _, loss_value = sess.run([train_op, loss], feed_dict={image_holder: image_batch, label_holder: label_batch})
    duration = time.time() - start_time

    if step % 10 == 0:
        examples_per_sec = batch_size / duration
        sec_per_batch = float(duration)

        print('step {},loss={},({} examples/sec; {} sec/batch)'.format(step, loss_value, examples_per_sec, sec_per_batch))

C:\anaconda\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


W0611 21:04:39.062251  6876 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.


step 0,loss=2.301417112350464,(250.668335078006 examples/sec; 0.5106348991394043 sec/batch)
step 10,loss=2.031980514526367,(345.93557607612155 examples/sec; 0.3700110912322998 sec/batch)
step 20,loss=1.798729658126831,(372.00654112873275 examples/sec; 0.34407997131347656 sec/batch)
step 30,loss=1.7974029779434204,(331.63435100169005 examples/sec; 0.3859672546386719 sec/batch)
step 40,loss=1.722352147102356,(453.5056832772439 examples/sec; 0.2822456359863281 sec/batch)
step 50,loss=1.749363899230957,(321.65948620366123 examples/sec; 0.3979363441467285 sec/batch)
step 60,loss=1.661386489868164,(329.9285241090095 examples/sec; 0.38796281814575195 sec/batch)
step 70,loss=1.5997768640518188,(361.526555695547 examples/sec; 0.35405421257019043 sec/batch)
step 80,loss=1.622209072113037,(406.1459375262413 examples/sec; 0.3151576519012451 sec/batch)
step 90,loss=1.4217431545257568,(407.43556075323124 examples/sec; 0.3141601085662842 sec/batch)
step 100,loss=1.4897708892822266,(361.52679914613367

step 850,loss=0.8817932605743408,(364.6086421480273 examples/sec; 0.3510613441467285 sec/batch)
step 860,loss=0.923652708530426,(375.2691382601639 examples/sec; 0.34108853340148926 sec/batch)
step 870,loss=0.7483811378479004,(381.97071138340084 examples/sec; 0.33510422706604004 sec/batch)
step 880,loss=0.9389533996582031,(402.32408290018765 examples/sec; 0.31815147399902344 sec/batch)
step 890,loss=0.9198735952377319,(367.74272369031223 examples/sec; 0.348069429397583 sec/batch)
step 900,loss=0.7090661525726318,(350.6613299334302 examples/sec; 0.3650245666503906 sec/batch)
step 910,loss=0.7282440662384033,(339.52954807047723 examples/sec; 0.37699222564697266 sec/batch)
step 920,loss=0.7978273630142212,(438.0277582190747 examples/sec; 0.2922189235687256 sec/batch)
step 930,loss=0.7819561958312988,(341.33550348601966 examples/sec; 0.374997615814209 sec/batch)
step 940,loss=0.7577551603317261,(332.4928667157165 examples/sec; 0.38497066497802734 sec/batch)
step 950,loss=0.8617343902587891,

step 1690,loss=0.7003335952758789,(362.55047855339313 examples/sec; 0.3530542850494385 sec/batch)
step 1700,loss=0.7494313716888428,(387.74133095865915 examples/sec; 0.33011698722839355 sec/batch)
step 1710,loss=0.6891968846321106,(330.77883073298375 examples/sec; 0.3869655132293701 sec/batch)
step 1720,loss=0.7045999765396118,(254.6476844493331 examples/sec; 0.5026552677154541 sec/batch)
step 1730,loss=0.5874133706092834,(336.8557415086609 examples/sec; 0.37998461723327637 sec/batch)
step 1740,loss=0.6209293007850647,(283.31591620909205 examples/sec; 0.45179247856140137 sec/batch)
step 1750,loss=0.5591658353805542,(361.52704259704825 examples/sec; 0.3540537357330322 sec/batch)
step 1760,loss=0.61259925365448,(350.6613299334302 examples/sec; 0.3650245666503906 sec/batch)
step 1770,loss=0.5734780430793762,(349.706333681821 examples/sec; 0.36602139472961426 sec/batch)
step 1780,loss=0.7780797481536865,(254.1428436853987 examples/sec; 0.5036537647247314 sec/batch)
step 1790,loss=0.5065039

step 2520,loss=0.49410223960876465,(264.62329284329945 examples/sec; 0.4837064743041992 sec/batch)
step 2530,loss=0.47274431586265564,(361.526555695547 examples/sec; 0.35405421257019043 sec/batch)
step 2540,loss=0.3632986545562744,(426.385480029322 examples/sec; 0.3001978397369385 sec/batch)
step 2550,loss=0.4290016293525696,(351.62201696181853 examples/sec; 0.3640272617340088 sec/batch)
step 2560,loss=0.4195009469985962,(248.24368687510895 examples/sec; 0.5156223773956299 sec/batch)
step 2570,loss=0.42601704597473145,(362.54827508431106 examples/sec; 0.3530564308166504 sec/batch)
step 2580,loss=0.35602521896362305,(282.6917418893509 examples/sec; 0.4527900218963623 sec/batch)
step 2590,loss=0.5489739179611206,(348.7567515598762 examples/sec; 0.3670179843902588 sec/batch)
step 2600,loss=0.3559521436691284,(348.7558453386376 examples/sec; 0.3670189380645752 sec/batch)
step 2610,loss=0.37680867314338684,(365.64638736915805 examples/sec; 0.3500649929046631 sec/batch)
step 2620,loss=0.3228

In [42]:
num_examples = 10000
num_iter = int(math.ceil(num_examples / batch_size))  # 计算一共有多少组
true_count = 0
total_sample_count = num_iter * batch_size
step = 0
while step < num_iter:
    image_batch, label_batch = sess.run([images_test, labels_test])
    predictions = sess.run([top_k_op], feed_dict={image_holder: image_batch, label_holder: label_batch})
    true_count += np.sum(predictions)
    step += 1

precision = true_count / total_sample_count
print('precision = ', precision)

precision =  0.8341574367088608
